#### 資料

In [ ]:
# get all data
!python label_studio.py --save_dir ./all_data --splits 1 0 0 --options labelstudio_data/label.txt

In [12]:
import os
import json


def collect_label_information(
    label_file: str = "/home/ubuntu/work_debug/verdict-cls-debug/utc/zero_shot_text_classification/data/label.txt",
):
    label_name = []
    label_count = {}
    with open(label_file, "r") as f:
        for i, name in enumerate(f):
            label_count[i] = 0
            label_name.append(name.strip())
    return label_name, label_count


currentdir = os.getcwd()
filedir = "zero_shot_text_classification/all_data/train.txt"
label_name, label_distribution = collect_label_information()
total_num_of_label = len(label_name)
all_data = []


dir = os.path.join(currentdir, filedir)
with open(dir, "r", encoding="utf8") as f:
    # tmp = f.readlines()
    for i in f:
        this_json = json.loads(i)
        all_data.append(this_json)
        for each_label in this_json["labels"]:
            label_distribution[each_label] += 1

print(len(all_data))
for each_label, each_name in zip(label_distribution, label_name):
    print(f"Label {each_label}, {each_name}: {label_distribution[each_label]}.")

def get_label_less_than(threshold = 10):
    print(f"------ All labels which cases are less than {threshold} are shown as follow. -----")
    result = {}
    for each_label, each_name in zip(label_distribution, label_name):
        if label_distribution[each_label] < threshold:
            print(f"Label: {each_label}, {each_name} is {label_distribution[each_label]}.")
            result[each_label] = (label_distribution[each_label], each_name)
    return result

              

875
Label 0, 頭頸部: 265.
Label 1, 臉: 173.
Label 2, 胸部: 186.
Label 3, 腹部: 117.
Label 4, 背部: 64.
Label 5, 骨盆: 92.
Label 6, 上肢: 426.
Label 7, 下肢: 513.
Label 8, 其他: 52.
Label 9, 骨折: 404.
Label 10, 骨裂: 3.
Label 11, 擦挫傷: 460.
Label 12, 撕裂傷: 138.
Label 13, 鈍傷: 59.
Label 14, 損傷: 100.
Label 15, 胸部損傷: 44.
Label 16, 神經損傷: 64.
Label 17, 拉傷: 19.
Label 18, 扭傷: 34.
Label 19, 灼傷: 6.
Label 20, 脫位: 68.
Label 21, 壓迫: 41.
Label 22, 破缺損: 112.
Label 23, 腦震盪: 62.
Label 24, 壞死: 8.
Label 25, 內出血: 113.
Label 26, 水腫: 0.
Label 27, 瘀血: 27.
Label 28, 栓塞: 1.
Label 29, 剝離: 0.
Label 30, 截肢: 6.
Label 31, 衰竭: 9.
Label 32, 休克: 10.
Label 33, 失能: 13.
Label 34, 死亡: 29.
Label 35, 其他: 0.
Label 36, 肇責 0/100: 96.
Label 37, 肇責 10/90: 11.
Label 38, 肇責 20/80: 39.
Label 39, 肇責 30/70: 106.
Label 40, 肇責 40/60: 33.
Label 41, 肇責 50/50: 52.
Label 42, 肇責 60/40: 47.
Label 43, 肇責 70/30: 49.
Label 44, 肇責 80/20: 8.
Label 45, 肇責 90/10: 2.
Label 46, 肇責 100/0: 15.
Label 47, 未滿18歲(高中以下): 10.
Label 48, 18-24歲(大學、研究所): 40.
Label 49, 25-29歲: 23.
Labe

In [70]:
# write data
data_type = ['train', 'dev', 'test']
def split_data(data, split_ratio=[0.8, 0.1, 0.1], folder_name: str = "remove_sparse_data"):
    p1 = int(len(data) * split_ratio[0])
    p2 = int(len(data) * split_ratio[1])
    output_data = (data[:p1], data[p1:p2], data[p2:])
    
    for each_data_type, out_data in zip(data_type, output_data):
        with open(os.path.join(os.getcwd(), "zero_shot_text_classification/", folder_name, each_data_type+ ".txt"), 'w', encoding='utf-8') as outfile:
            jsonString = json.dumps(out_data, ensure_ascii=False, default=int)
            outfile.write(jsonString)

#### Case 1. 類別少於 Threshold 則刪除該類別

In [46]:
remove_set = get_label_less_than(10)
remove_set[35] = (0, "其他受傷程度")
print(remove_set)

------ All labels which cases are less than 10 are shown as follow. -----
Label: 10, 骨裂 is 3.
Label: 19, 灼傷 is 6.
Label: 24, 壞死 is 8.
Label: 26, 水腫 is 0.
Label: 28, 栓塞 is 1.
Label: 29, 剝離 is 0.
Label: 30, 截肢 is 6.
Label: 31, 衰竭 is 9.
Label: 35, 其他 is 0.
Label: 44, 肇責 80/20 is 8.
Label: 45, 肇責 90/10 is 2.
{10: (3, '骨裂'), 19: (6, '灼傷'), 24: (8, '壞死'), 26: (0, '水腫'), 28: (1, '栓塞'), 29: (0, '剝離'), 30: (6, '截肢'), 31: (9, '衰竭'), 35: (0, '其他受傷程度'), 44: (8, '肇責 80/20'), 45: (2, '肇責 90/10')}


In [49]:
import numpy as np
from collections import defaultdict
def remove_sparse_data():
    new_label_distribution = defaultdict(int)
    result = []
    with open(os.path.join(os.getcwd(), "zero_shot_text_classification/all_data/train.txt")) as f:
        for i in f:
            example = json.loads(i)
            example['choices'][8] = "其他體傷部位"
            example['choices'][35] = "其他受傷程度"
            this_label = []
            
            for label in example['labels']:
                this_label.append(example['choices'][label])        

            # remove label
            for remove_label, remove_name in remove_set.items():
                remove_name = remove_name[1]
                pop_index = np.where(np.array(example['choices']) == remove_name)[0][0]
                example['choices'].pop(pop_index)
                 
                if remove_name in this_label:
                    pop_index = np.where(np.array(this_label) == remove_name)[0][0]
                    this_label.pop(pop_index)

            # add new label
            new_label = []
            
            for label in this_label:
                label_index = np.where(np.array(example['choices']) == label)[0][0]
                new_label_distribution[label_index] += 1
                new_label.append(label_index)
            example['labels'] = new_label
            result.append(example)
    return result, new_label_distribution


remove_name = [label_name for _, label_name in remove_set.values()]


new_data, new_label_distribution = remove_sparse_data()

In [50]:
new_label_distribution
sorted(new_label_distribution.items(), key=lambda x: x[0])

[(0, 265),
 (1, 173),
 (2, 186),
 (3, 117),
 (4, 64),
 (5, 92),
 (6, 426),
 (7, 513),
 (8, 52),
 (9, 404),
 (10, 460),
 (11, 138),
 (12, 59),
 (13, 100),
 (14, 44),
 (15, 64),
 (16, 19),
 (17, 34),
 (18, 68),
 (19, 41),
 (20, 112),
 (21, 62),
 (22, 113),
 (23, 27),
 (24, 10),
 (25, 13),
 (26, 29),
 (27, 96),
 (28, 11),
 (29, 39),
 (30, 106),
 (31, 33),
 (32, 52),
 (33, 47),
 (34, 49),
 (35, 15),
 (36, 10),
 (37, 40),
 (38, 23),
 (39, 38),
 (40, 40),
 (41, 41),
 (42, 15),
 (43, 48)]

In [71]:
split_data(new_data)
# text = "\n".join([str({'b':2}),str({'a':1})])
# with open('try.txt','w') as f:
#    f.write(text)

#### Case 2. 類別合併

#### Case 3. 複製少量類別的樣本